# File Search Store Management for Rickbot

A notebook to experiment with the FileSearchStore and how it can be used to manage file search in the Rickbot Agent.

The best way to run this notebook is from Google Colab.

<a target="_blank" href="https://colab.research.google.com/github/derailed-dash/rickbot-adk/blob/feat/file-search-store/notebooks/file_search_store.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" height=30/></a>

## Pre-Reqs and Notes

- The `file_search_stores` is a feature exclusive to the Gemini Developer API. 
  - It does not work with the Vertex AI API or the Gen AI SDK in Vertex AI mode.
  - Therefore: don't set env vars for `GOOGLE_CLOUD_LOCATION` or `GOOGLE_GENAI_USE_VERTEXAI` and do not initialise Vertex AI.
- Make sure you have an up-to-date version of the `google-genai` package installed. 
  - Versions older than 1.49.0 do not support the File Search Tool.
  - You can upgrade the package using `pip install --upgrade google-genai`.
  - You can add to your `pyproject.toml` file; since we don't explicitly need it outside of this notebook, we can add it to the `[jupyter]` section.
- Add your Gemini API Key to Colab as a secret. Then you can retrieve it using `userdata.get("GOOGLE_API_KEY")`

## Setup

### Local Only

If running locally, setup the Google Cloud environment:

```bash
source scripts/setup-env.sh
```

Then to install the package dependencies into the virtual environment, use the `uv` tool:

1. From your agent's root directory, run `make install` to set up the virtual environment (`.venv`).
2. In this Jupyter notebook, select the kernel from the `.venv` folder to ensure all dependencies are available.

In [ ]:
import os

from dotenv import load_dotenv

if load_dotenv(".env.local"):
    print("Loaded env")
else:
    print("Warning: .env file not found")

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
print(f"Using Google API key ending: {GEMINI_API_KEY[:10]}")
APP_NAME = "rickbot_notebook_client"

Loaded env
Using Google API key ending: AIzaSyDguo


### Or In Colab

In [16]:
%pip install -q -U "google-genai>=1.49.0"

/home/darren/localdev/python/rickbot-adk/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from google.colab import userdata

os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

### Then

In [ ]:
import os

from google import genai
from google.genai.types import Blob, Content, Part
from IPython.display import Markdown, display


### Client Initialisation

In [5]:
client = genai.Client()

## Store Management

### Create the Store (One Time)

In [6]:
DAZBO_STORE_NAME = "rickbot-dazbo-ref"
file_search_store = client.file_search_stores.create(config={'display_name': DAZBO_STORE_NAME})

### View Stores

In [10]:
for a_store in client.file_search_stores.list():
    print(a_store)

name='fileSearchStores/rickbotdazboref-akasbs0scqrs' display_name='rickbot-dazbo-ref' create_time=datetime.datetime(2025, 12, 31, 16, 41, 49, 4605, tzinfo=TzInfo(0)) update_time=datetime.datetime(2025, 12, 31, 16, 41, 49, 4605, tzinfo=TzInfo(0)) active_documents_count=None pending_documents_count=None failed_documents_count=None size_bytes=None


## Upload and Process Files

Now we need to place the files we will use in a suitable folder. Let's go with...